# **Модель, распознающая эмоции**
Здесь описано создание и обучение модели, распознающей эмоции по фотографии.

## Эмоции
Модель распознает 7 базовых эмоций по Экману:
- злость (ANGER)
- отвращение (DISGUST)
- страх (FEAR)
- счастье (HAPPINESS)
- грусть (SAD)
- удивление (SURPRISE)
- нейтральное выражение (NEUTRAL)

Инициализируем массив эмоций

In [0]:
emotions = [(0, "ANGER"), (1, "DISGUST"), (2, "FEAR"), (3, "HAPPINESS"), 
            (4, "SAD"), (5, "SURPRISE"), (6, "NEUTRAL")]

## Датасет
Для обучения, будем использовать датасет fer2013. Найти его можно [здесь](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data). Загрузите его и поместите в папку с этим файлом.
Или можно примонтировать Google Drive, если используется Google Colab.

In [34]:
# Монтирование Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Путь до датасета, укажите свой
fer_dataset_path = "/content/drive/My Drive/university/reko/fer2013.csv"

## Подготовка данных
Для обучения подготавливаем данные. Грузим датасет. Изображения в датасете имеют размер 48х48 пикселей

In [0]:
import pandas as pd
import numpy as np

In [0]:
width, height = 48, 48
data = pd.read_csv(fer_dataset_path)

Данные в датасете хранятся в виде векторов целых чисел. В столбце emotion указан идентификатор эмоции (ассоциативный массив определен выше), а pixels - это 2304 (48х48) чисел изображения, пиксели. Пример как это хранится внутри csv файла ниже:

In [23]:
data[:5]

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [0]:
datapoints = data['pixels'].tolist()

all_images = []
for image_xs in datapoints:
  # преобразуем из строки в массив чисел
  xx = [int(image_x) for image_x in image_xs.split(' ')]
  # преобразуем одномерный массив в матрицу
  xx = np.asarray(xx).reshape(width, height)
  # добавляем изображение в список всех изображений
  all_images.append(xx.astype('float32'))

In [0]:
# преобразование к массиву numpy
all_images = np.asarray(all_images)
all_images = np.expand_dims(all_images, -1)

Считываем эмоции:

In [26]:
all_emotions = pd.get_dummies(data['emotion'])
np.array(all_emotions)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=uint8)

## Модель
Для распознавания эмоций на изображении лучше всего использовать сверточную нейронную сеть. Для построения модели будем использовать Keras и TensorFlow. Делаем нужные импорты

In [0]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2

import tensorflow as tf

Так же задаем нужные константы

In [0]:
num_features = 64 # количество признаков
num_labels = len(emotions) # количество эмоций
batch_size = 64 # каким размером партии данных обучать
epochs = 100 # количество эпох, сколько раз обучать

Разделяем данные для теста и обучения

In [29]:
separate_index = int(len(all_images) * 0.9)
test_images, train_images = all_images[separate_index:], all_images[:separate_index]
test_emotions, train_emotions = all_emotions[separate_index:], all_emotions[:separate_index]

print("Размер тестовой подборки: {0}".format(len(test_images)))
print("Размер обучающей подборки: {0}".format(len(train_images)))

Размер тестовой подборки: 3589
Размер обучающей подборки: 32298


Сама сверточная нейронная сеть построена следующим образом

In [30]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_8 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_9 (Batch (None, 23, 23, 128)      

Модель создана, теперь следует её скопилировать и обучить.

In [0]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [32]:
model.fit(train_images, train_emotions,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_images, test_emotions),
          shuffle=True)

Train on 32298 samples, validate on 3589 samples
Epoch 1/100
32298/32298 [==============================] - 80s 2ms/step - loss: 2.0393 - accuracy: 0.2144 - val_loss: 1.8380 - val_accuracy: 0.2452
Epoch 2/100
32298/32298 [==============================] - 78s 2ms/step - loss: 1.8358 - accuracy: 0.2475 - val_loss: 1.8243 - val_accuracy: 0.2449
Epoch 3/100
32298/32298 [==============================] - 78s 2ms/step - loss: 1.7979 - accuracy: 0.2679 - val_loss: 1.7554 - val_accuracy: 0.2917
Epoch 4/100
32298/32298 [==============================] - 78s 2ms/step - loss: 1.7117 - accuracy: 0.3074 - val_loss: 2.0654 - val_accuracy: 0.1948
Epoch 5/100
32298/32298 [==============================] - 78s 2ms/step - loss: 1.5838 - accuracy: 0.3707 - val_loss: 1.5552 - val_accuracy: 0.3918
Epoch 6/100
32298/32298 [==============================] - 77s 2ms/step - loss: 1.4993 - accuracy: 0.4152 - val_loss: 1.4685 - val_accuracy: 0.4361
Epoch 7/100
32298/32298 [==============================] - 78s 

Проверяем работоспособность модели:

In [0]:
def get_index_of_max(arr):
  return np.where(arr == max(arr))[0][0]

def get_emotion(index):
  for id, emotion in emotions:
    if (id == index):
      return emotion

In [83]:
image = np.array([test_images[2]])
predicted = model.predict(image)[0]
get_emotion(get_index_of_max(predicted))

'FEAR'

Сохраняем модель

In [0]:
model.save("fer_model.tf")

Загружаем модель с диска (если она у вас не загружена)

In [0]:
# Укажите свой путь до обученной модели
model_path = "/content/drive/My Drive/university/reko/fer_model.tf"
model = load_model(model_path)

Так же можно конвертировать Keras модель в tflite.

In [52]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open("model.tflite", "wb").write(tfmodel)

AttributeError: ignored